In [1]:
import polars as pl
import polars.selectors as cs
from pathlib import Path

lf_list = []

for path in Path("output").glob("*.parquet"):
  lf_list.append(pl.scan_parquet(path))

lf: pl.LazyFrame = pl.concat(lf_list).with_columns(
  (pl.col("meteor") - pl.col("meteor_shot")).alias("remaining")
)

lf.drop("time", "id").describe()


statistic,meteor,mdi_ready,shot_fired,meteor_shot,remaining
str,f64,f64,f64,f64,f64
"""count""",7.0065582e7,7.0065582e7,7.0065582e7,7.0065582e7,7.0065582e7
"""null_count""",0.0,0.0,0.0,0.0,0.0
"""mean""",1.999794,23.904229,2.499683,1.999779,0.000015
"""std""",1.414185,0.611644,1.936115,1.414056,0.006812
"""min""",1.0,0.0,1.0,0.0,0.0
"""25%""",1.0,24.0,1.0,1.0,0.0
"""50%""",1.0,24.0,2.0,1.0,0.0
"""75%""",2.0,24.0,3.0,2.0,0.0
"""max""",27.0,24.0,24.0,23.0,13.0


In [2]:
lf_2 = (
  lf.drop("time")
  .group_by("id")
  .agg(cs.exclude("mdi_ready").sum().name.suffix("_sum"))
  .drop("id")
)

lf_2.describe()


statistic,meteor_sum,shot_fired_sum,meteor_shot_sum,remaining_sum
str,f64,f64,f64,f64
"""count""",20000.0,20000.0,20000.0,20000.0
"""null_count""",0.0,0.0,0.0,0.0
"""mean""",7005.8352,8757.0856,7005.78345,0.05175
"""std""",105.385057,140.16681,105.379617,0.403956
"""min""",6613.0,8110.0,6613.0,0.0
"""25%""",6935.0,8662.0,6935.0,0.0
"""50%""",7006.0,8756.0,7006.0,0.0
"""75%""",7076.0,8851.0,7076.0,0.0
"""max""",7424.0,9324.0,7424.0,13.0


In [3]:
lf_2.group_by("remaining_sum").len().sort("remaining_sum").collect()


remaining_sum,len
i64,u32
0,19502
1,249
2,118
3,53
4,34
5,27
6,9
7,5
9,2
